In [1]:
import psycopg2
from age.age_networkx import *
import random
import time

# set DB path and graph name
conn = psycopg2.connect(
    host="localhost",
    port="5430",
    dbname="postgresDB",
    user="postgresUser",
    password="postgresPW")
GRAPH_NAME = 'bitnine_global_inic'

## Networkx to AGE

### Creating a Random Networkx Directed Graph

In [2]:
number_of_nodes = 4000
number_of_edges = 8000

# Create an empty graph
G = nx.DiGraph()

for i in range(number_of_nodes):
    G.add_node(i,
               label='People',
               properties={'id': i, 'weight': i*2+20})

for i in range(number_of_edges):
    a = 0
    b = 0
    while(a==b):
        a = random_number = random.randint(0, number_of_nodes-1)
        b = random_number = random.randint(0, number_of_nodes-1)
    
    G.add_edge(a, b, label='Connection',
        properties={'firstMeet': 'Jan-21-2019', 'start_id' : a, 'end_id' : b})
print(G)

DiGraph with 4000 nodes and 8000 edges


### networkx to AGE in a newly created graph

In [3]:
age.setUpAge(conn, GRAPH_NAME)
age.deleteGraph(conn, GRAPH_NAME)
age.setUpAge(conn, GRAPH_NAME)

start_time = time.perf_counter()

networkxToAge(conn, G, GRAPH_NAME=GRAPH_NAME)

now_time = time.perf_counter()
elapsed_time_ms = (now_time - start_time)
print(f'{elapsed_time_ms}s')

500  nodes created
1000  nodes created
1500  nodes created
2000  nodes created
2500  nodes created
3000  nodes created
3500  nodes created
4000  nodes created
ADDED : 1000  Edges
ADDED : 2000  Edges
ADDED : 3000  Edges
ADDED : 4000  Edges
ADDED : 5000  Edges
ADDED : 6000  Edges
ADDED : 7000  Edges
ADDED : 8000  Edges
7.994979198003421s


## AGE to Networkx

In [4]:
start_time = time.perf_counter()

G = ageToNetworkx(conn,GRAPH_NAME)
print (G)

now_time = time.perf_counter()
elapsed_time_ms = (now_time - start_time)
print(f'{elapsed_time_ms}s')

DiGraph with 4000 nodes and 8000 edges
16.543609005995677s
